In [19]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

sys.path.append(r'C:\Users\Pooya\w\GitHub\ShiraziPooya\DroughtMonitoringIran')

### Load All Data

In [20]:
DATABASE_PATH = "../database/database.db"

conn = sqlite3.connect(DATABASE_PATH)

ground_pet_data = pd.read_sql(sql='SELECT * FROM ground_pet_monthly', con=conn)
ground_pet_data['date'] = pd.to_datetime(ground_pet_data['date'])
ground_pet_data['date'] = ground_pet_data['date'].dt.to_period('M').astype(str)
ground_pet_data['station_id'] = ground_pet_data['station_id'].astype(str)
ground_pet_data.drop(
    columns=['year', 'month', 'lat', 'lon', 'station_elevation', 'tmax_count', 'tmin_count', 'tm_count', 'rrr24_count'],
    inplace=True
)

ground_di_data = pd.read_sql(sql='SELECT * FROM ground_di_monthly', con=conn)
ground_di_data['date'] = pd.to_datetime(ground_di_data['date'])
ground_di_data['date'] = ground_di_data['date'].dt.to_period('M').astype(str)
ground_di_data['station_id'] = ground_di_data['station_id'].astype(str)
ground_di_data.drop(
    columns=['lat', 'lon', 'station_elevation'],
    inplace=True
)

gee_precip_data = pd.read_sql(sql='SELECT * FROM gee_precip_monthly', con=conn)
gee_precip_data['date'] = pd.to_datetime(gee_precip_data['date'])
gee_precip_data['date'] = gee_precip_data['date'].dt.to_period('M').astype(str)
gee_precip_data['station_id'] = gee_precip_data['station_id'].astype(str)
gee_precip_data.drop(
    columns=['lat', 'lon', 'station_elevation'],
    inplace=True
)

gee_pet_data = pd.read_sql(sql='SELECT * FROM gee_pet_monthly', con=conn)
gee_pet_data['date'] = pd.to_datetime(gee_pet_data['date'])
gee_pet_data['date'] = gee_pet_data['date'].dt.to_period("M").astype(str)
gee_pet_data['station_id'] = gee_pet_data['station_id'].astype(str)
gee_pet_data.drop(
    columns=['lat', 'lon', 'station_elevation'],
    inplace=True
)

gee_indices_data = pd.read_sql(sql='SELECT * FROM gee_indices_monthly', con=conn)
gee_indices_data['date'] = pd.to_datetime(gee_indices_data['date'])
gee_indices_data['date'] = gee_indices_data['date'].dt.to_period("M").astype(str)
gee_indices_data['station_id'] = gee_indices_data['station_id'].astype(str)

gee_di_data = pd.read_sql(sql='SELECT * FROM gee_di_monthly', con=conn)
gee_di_data['date'] = pd.to_datetime(gee_di_data['date'])
gee_di_data['date'] = gee_di_data['date'].dt.to_period("M").astype(str)
gee_di_data['station_id'] = gee_di_data['station_id'].astype(str)

ground_geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)
ground_geoinfo['station_id'] = ground_geoinfo['station_id'].astype(str)

conn.close()

ground_geoinfo

,region_id,region_name,station_name,station_id,lat,lon,station_elevation
0,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0
1,MASA,Mazandaran,Amol,99309,36.48,52.47,23.7
2,MASA,Mazandaran,Babolsar,40736,36.70,52.64,-21.0
3,MASA,Mazandaran,Baladeh,99357,36.20,51.80,2120.0
4,MASA,Mazandaran,Bandar-E-Amirabad,99306,36.86,53.39,-20.0
5,MASA,Mazandaran,Galugah,99299,36.74,53.84,-10.0
6,MASA,Mazandaran,Gharakhil,40737,36.49,52.11,14.7
7,MASA,Mazandaran,Kiyasar,40760,36.25,53.55,1294.3
8,MASA,Mazandaran,Kojur,99348,36.39,51.73,1550.0
9,MASA,Mazandaran,Nowshahr,40734,36.66,51.47,-20.9


In [26]:
data = ground_geoinfo.copy()

data = data\
    .merge(ground_pet_data, on=['region_id', 'region_name', 'station_id', 'station_name'], how='outer')\
    .merge(ground_di_data, on=['region_id', 'region_name', 'station_id', 'station_name', 'date'], how='outer')\
    .merge(gee_precip_data, on=['region_id', 'region_name', 'station_id', 'station_name', 'date'], how='outer')\
    .merge(gee_pet_data, on=['region_id', 'region_name', 'station_id', 'station_name', 'date'], how='outer')\
    .merge(gee_indices_data, on=['region_id', 'region_name', 'station_id', 'station_name', 'date'], how='outer')\
    .merge(gee_di_data, on=['region_id', 'region_name', 'station_id', 'station_name', 'date'], how='outer')\
    .sort_values(['region_id', 'region_name', 'station_id', 'station_name', 'date'])\
    .query('date >= "1980-01" and date <= "2025-09"')\
    .reset_index(drop=True)

data = data.round(3)

data

,region_id,region_name,station_name,station_id,lat,lon,station_elevation,date,tmax,tmin,...,ERA5_SPI_24,ERA5_SPEI_24,GPM_SPI_24,GPM_SPEI_24,PERSIANN_CDR_SPI_24,PERSIANN_CDR_SPEI_24,TRMM_SPI_24,TRMM_SPEI_24,TerraClimate_SPI_24,TerraClimate_SPEI_24
0,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-01,9.55,1.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-02,8.46,2.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-03,10.53,5.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-04,15.83,10.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-05,20.67,14.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8230,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0,2025-05,22.36,11.35,...,-0.960,NaN,-1.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8231,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0,2025-06,20.64,11.66,...,-1.353,NaN,-1.689,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8232,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0,2025-07,24.43,15.14,...,-0.881,NaN,-1.555,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8233,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0,2025-08,26.52,16.82,...,-1.584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8235 entries, 0 to 8234
Columns: 178 entries, region_id to TerraClimate_SPEI_24
dtypes: float64(173), object(5)
memory usage: 11.2+ MB


In [28]:
conn = sqlite3.connect(database=DATABASE_PATH)
data.to_sql(name='data', con=conn, if_exists='replace', index=False)
conn.commit()
conn.close()